In [1]:
import pct
from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass

### Model Training Setup:

In [5]:
@dataclass
class TrainConfig:
    """Training paramaters for training of pct.
    """
    epochs: int = 10
    batch_size: int = 64
    lr: float = 1e-4
    d_model: int = 128
    depth: int = 3
    n_heads: int = 4
    k: int = 3
    latent_dim: int = 128
    out_dim: int = 1
    drop: float = 0.0

### Create Dummy Training Data
* Creates a sphere of point cloud data to enable demonstration of pct operation.
* Creates PyTorch dataloaders with specified batch_size for ```train``` and ```val```.

In [3]:
train_ds = pct.DummyPointCloudDataset(n_samples=800, n_points=512)
val_ds = pct.DummyPointCloudDataset(n_samples=200, n_points=512)

train_loader = DataLoader(train_ds, batch_size=TrainConfig.batch_size, shuffle=True, num_workers=0, collate_fn=pct.collate_batch)
val_loader = DataLoader(val_ds, batch_size=TrainConfig.batch_size, shuffle=False, num_workers=0, collate_fn=pct.collate_batch)

In [6]:
model = pct.PointCloudTransformer(
        in_channels=3,
        d_model=TrainConfig.d_model,
        depth=TrainConfig.depth,
        n_heads=TrainConfig.n_heads,
        k=TrainConfig.k,
        latent_dim=TrainConfig.latent_dim,
        out_dim=TrainConfig.out_dim,
        drop=TrainConfig.drop,
    )
model

PointCloudTransformer(
  (encoder): PointTransformerEncoder(
    (input_proj): Sequential(
      (0): Linear(in_features=3, out_features=128, bias=True)
      (1): GELU(approximate='none')
      (2): Linear(in_features=128, out_features=128, bias=True)
    )
    (blocks): ModuleList(
      (0-2): 3 x PointTransformerBlock(
        (norm_q): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (q_proj): Linear(in_features=128, out_features=128, bias=True)
        (k_proj): Linear(in_features=128, out_features=128, bias=True)
        (v_proj): Linear(in_features=128, out_features=128, bias=True)
        (rel_enc): RelPosEncoding(
          (mlp): Sequential(
            (0): Linear(in_features=4, out_features=64, bias=True)
            (1): GELU(approximate='none')
            (2): Linear(in_features=64, out_features=128, bias=True)
          )
        )
        (attn_out): Linear(in_features=128, out_features=128, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
       

### Model Training:
* The model is trained using the ```train_loader``` and ```val_loader```.
* Epochs and learning rate are specified in the ```TrainConfig``` dataclass.

In [7]:
pct.train_regression(model, train_loader, val_loader, epochs=TrainConfig.epochs, lr=TrainConfig.lr)

/Users/joshuahellewell/miniforge3/envs/pct/lib/python3.13/site-packages/pct/trainer.py:19: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device.startswith('cuda')))
/Users/joshuahellewell/miniforge3/envs/pct/lib/python3.13/site-packages/pct/trainer.py:29: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device.startswith('cuda'))):
/Users/joshuahellewell/miniforge3/envs/pct/lib/python3.13/site-packages/pct/trainer.py:45: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device.startswith('cuda'))):


Epoch 001 | train 0.0650 | val 0.0027
Epoch 002 | train 0.0007 | val 0.0002
Epoch 003 | train 0.0002 | val 0.0001
Epoch 004 | train 0.0001 | val 0.0001
Epoch 005 | train 0.0001 | val 0.0002
Epoch 006 | train 0.0002 | val 0.0001
Epoch 007 | train 0.0001 | val 0.0001
Epoch 008 | train 0.0001 | val 0.0001
Epoch 009 | train 0.0001 | val 0.0001
Epoch 010 | train 0.0001 | val 0.0002


### Extract Latent Vectors

In [ ]:
train_ds

In [12]:
latents = pct.extract_latents(train_loader,model)
print(latents.shape)

torch.Size([800, 128])


In [18]:
points, targets = train_ds[2]

pct.visualize_point_cloud(points, title=f"Noisy Sphere (r={targets.item():.2f})")